In [ ]:
#HH[0,2000] by CNS

In [ ]:
function accuracy()
    ac=1200;
    return ac;
end
setprecision(accuracy());

In [ ]:
function order()
    M=70;
    return M+1;
end

In [ ]:
function steps()
    h = BigFloat(1)/BigFloat(10)
    return h;
end

In [ ]:
function multi()
    S = Array{BigFloat}(undef,1,order());
    S[1] = BigFloat(1)
    
    for i in 1:order()-1
        S[i+1] = S[i]*steps()
    end
    
    return S
end

In [ ]:
function multi2(Dmulti)
    S = Array{BigFloat}(undef,order()-1)
    for i in 1:order()-1
        S[i] = BigFloat(i)
    end
    return S.*Dmulti[1:end-1]
end

In [ ]:
"""fdot(x,y) = @. x*y
function Sumxyz(x,y)
    return sum(fdot(reverse!(x),y))
end
function Sumxyz2(x,y)
    return sum(fdot(reverse!(x),x) - fdot(reverse!(y),y))
end 
"""

In [ ]:
function Sumxyz(a,b)
    return sum(reverse!(b[1:end]).*a[1:end])
end
function Sumxyz2(a,b)
    return sum(a[1:end].*reverse!(a[1:end]) .- b[1:end].*reverse!(b[1:end]))
end


In [ ]:
function Talor(a,b)
    for n in 1:order()-2
        a[n+2] = -(a[n] + BigFloat(2)*Sumxyz(a[1:n],b[1:n]))/BigFloat((n+1)*n)
        b[n+2] = -(b[n] + Sumxyz2(a[1:n],b[1:n]))/BigFloat((n+1)*n)
    end
    return a,b
end

In [ ]:
function Iteration(a,b)
    
    for st in 1:Times
    
        a,b = Talor(a,b)
        
        a[1] = sum(a[1:end].*Dmulti[1:end])
        b[1] = sum(b[1:end].*Dmulti[1:end])
        a[2] = sum(a[2:end].*Dmulti2[1:end])
        b[2] = sum(b[2:end].*Dmulti2[1:end])
    
        Lorenz[st,:] = [a[1],b[1]] 
        Lorenzf[st,:] = convert.(Float32,Lorenz[st,:])
      
        if mod(st,1000) == 0
            println(st,")","Time=",convert(Float16,st*steps()),": ",Lorenzf[st,:])
        end
    end
    
    return Lorenz
    
end

In [ ]:
Times = 10*2000;      

a = Array{BigFloat}(undef,order())
b = Array{BigFloat}(undef,order())
Lorenz = Array{BigFloat}(undef,Times,2);
Lorenzf = Array{Float32}(undef,Times,2);
Dmulti = Array{BigFloat}(undef,1,order());
Dmulti2 = Array{BigFloat}(undef,1,order()-1);

a[1] = BigFloat(14)/BigFloat(25); 
b[1] = BigFloat(0); 
a[2] = BigFloat(0); 
b[2] = BigFloat(0); 

Dmulti = multi();
Dmulti2 = multi2(Dmulti);

In [ ]:
HH = Iteration(a,b);

In [ ]:
using Plots; 
plot(HH[1:end,1],HH[1:end,2])

In [ ]:
#write
function Wfile(AT,BE,Times)
#change name--------------------------------------------
    XLSX.openxlsx("CNS_HH.xlsx", mode="w") do xf
#-------------------------------------------------------
        sheet = xf[1]
        XLSX.rename!(sheet, "new_sheet")
#change location----------------------------------------
        sheet["A1"] = "T"
        sheet["B1"] = "X"
        sheet["A2",dim=1] = AT[1:end]
        sheet["B2",dim=1] = BE[1:end]
#-------------------------------------------------------
    end
end
#-----------------------------------------------------------------------------
using XLSX
#time-----------------------------------------------------
AT=Array{Float64}(undef,Times)
for i in 1:Times
    AT[i] = i*steps()
end
#time-----------------------------------------------------
BE = HH[:,1]
#--------------------------------------------------------
Wfile(AT,BE,Times);